In [3]:
from dotenv import load_dotenv
import os


load_dotenv()


os.environ['LANGSMITH_PROJECT']  ="skn15-1"


from langchain_openai import ChatOpenAI


llm = ChatOpenAI()
llm.invoke("Hello, world!")


AIMessage(content='Hello there! How can I assist you today?', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 10, 'prompt_tokens': 11, 'total_tokens': 21, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'id': 'chatcmpl-C0iDQGxDvJ0BZbOqo6d9cKCEdK8LQ', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None}, id='run--9208b8b6-92e6-44e2-a8c8-76f7b1641779-0', usage_metadata={'input_tokens': 11, 'output_tokens': 10, 'total_tokens': 21, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})

In [4]:
os.environ['LANGSMITH_PROJECT']

'skn15-1'

In [3]:
from langchain_community.document_loaders.csv_loader import CSVLoader
loader = CSVLoader(file_path="./아리계곡_통합.csv")
docs = loader.load()


In [4]:
from langchain_chroma import Chroma
from langchain_openai import OpenAIEmbeddings

embedding = OpenAIEmbeddings()
db = Chroma.from_documents(docs, embedding = embedding)


In [5]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough
from langchain_openai import ChatOpenAI


prompt = ChatPromptTemplate.from_template('''\
다음 문맥만을 고려해 질문에 답하세요.


문맥: """
{context}
"""


질문: {question}
''')


model = ChatOpenAI(model="gpt-4o-mini", temperature=0)


retriever = db.as_retriever()


chain = {
    "question": RunnablePassthrough(),
    "context": retriever,
} | prompt | model | StrOutputParser()


output = chain.invoke("가게 분위기는 어떤가?")


In [6]:
import pprint


pprint.pprint(output)


('가게 분위기는 전반적으로 좋다고 평가되고 있습니다. 여러 리뷰에서 "분위기 완전 좋고", "분위기도 좋고 깨끗해서", "분위기가 '
 '좋아요", "분위기도 좋고 음식도 매장 디자인도 괜찮고"라는 긍정적인 언급이 있습니다. 따라서 가게의 분위기는 매우 긍정적으로 '
 '받아들여지고 있습니다.')


In [12]:
os.environ['LANGSMITH_PROJECT']

'skn15-1'

In [15]:
from langchain_openai import ChatOpenAI


llm = ChatOpenAI()
llm.invoke("Hello, world!")


AIMessage(content='Hello! How can I assist you today?', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 9, 'prompt_tokens': 11, 'total_tokens': 20, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'id': 'chatcmpl-C0i9cevhzNaqrbH399kEgFtxdgNvd', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None}, id='run--82cbf119-6337-47b9-be68-86f44b446aa2-0', usage_metadata={'input_tokens': 11, 'output_tokens': 9, 'total_tokens': 20, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})

In [6]:
from pydantic import BaseModel, Field




class QueryGenerationOutput(BaseModel):
    queries: list[str] = Field(..., description="검색 쿼리 목록")




query_generation_prompt = ChatPromptTemplate.from_template("""\
질문에 대해 벡터 데이터베이스에서 관련 문서를 검색하기 위한
3개의 서로 다른 검색 쿼리를 생성하세요.
거리 기반 유사성 검색의 한계를 극복하기 위해
사용자의 질문에 대해 여러 관점을 제공하는 것이 목표입니다.


질문: {question}
""")


query_generation_chain = (
    query_generation_prompt
    | model.with_structured_output(QueryGenerationOutput)
    | (lambda x: x.queries)
)




multi_query_rag_chain = {
    "question": RunnablePassthrough(),
    "context": query_generation_chain | retriever.map(),
} | prompt | model | StrOutputParser()




multi_query_rag_chain.invoke("주위사람에게 추천할 의사는?")



NameError: name 'ChatPromptTemplate' is not defined

In [ ]:
from langchain_core.documents import Document




def reciprocal_rank_fusion( retriever_outputs: list[list[Document]],  k: int = 60 ) -> list[str]:
    # 각 문서의 콘텐츠(문자열)와 그 점수의 매핑을 저장하는 딕셔너리 준비
    content_score_mapping = {}


    # 검색 쿼리마다 반복
    for docs in retriever_outputs:
        # 검색 결과의 문서마다 반복
        for rank, doc in enumerate(docs):
            content = doc.page_content


            # 처음 등장한 콘텐츠인 경우 점수를 0으로 초기화
            if content not in content_score_mapping:
                content_score_mapping[content] = 0


            # (1 / (순위 + k)) 점수를 추가
            content_score_mapping[content] += 1 / (rank + k)


    # 점수가 큰 순서로 정렬
    ranked = sorted(content_score_mapping.items(), key=lambda x: x[1], reverse=True)  # noqa
    return [content for content, _ in ranked]


rag_fusion_chain = {
    "question": RunnablePassthrough(),
    "context": query_generation_chain | retriever.map() | reciprocal_rank_fusion,
} | prompt | model | StrOutputParser()



rag_fusion_chain.invoke("가게 분위기는?")

In [ ]:
from langchain_community.retrievers import TavilySearchAPIRetriever





langchain_document_retriever = retriever.with_config({'run_name' : 'langchain_document_retriver'})
web_retriever = TavilySearchAPIRetriever(k=10).with_config({'run_name' : 'web_retriver'})


from enum import Enum
from pydantic import BaseModel, Field


class Route(str, Enum):
    langchain_document = 'langchain_document'
    web = "web"


class RouteOutput(BaseModel):
    route: Route


route_prompt  =  ChatPromptTemplate.from_template("""  
질문에 답변하기 위한 적절한 Retriever를 선택하세요.


질문: {question}
""")


route_chain = (
    route_prompt | model.with_structured_output(RouteOutput)
    | (lambda x : x.route)
)




def routed_retriever(inp):
    question = inp['question']
    route = inp['route']


    if route == Route.langchain_document:
        return langchain_document_retriever.invoke(question)
    elif route == Route.web:
        return web_retriever.invoke(question)


    raise ValueError(f"Unkown route: {route}")




route_rag_chain = ({
    'question' : RunnablePassthrough(),
    "route" : route_chain
}
| RunnablePassthrough.assign(context=routed_retriever)
| prompt | model | StrOutputParser())




route_rag_chain.invoke("아리계곡 마감 시간은?")




In [ ]:
import re
import requests
from bs4 import BeautifulSoup




import re 
p = re.compile("'([0-9]+)'")




def get_ticker(company):
    text_url = "https://dart.fss.or.kr/dsae001/search.ax"
    payload = {
    "startDate": "",     "endDate": "",
    "currentPage": 1,     "maxResults": 45,
    "maxLinks": 10,     "sort": "",
    "series": "",     "selectKey": "",
    "searchIndex": "",     "textCrpCik": "",
    "autoSearch": True,     "businessCode": "all",
    "bsnRgsNo": "",     "bsnRgsNo_1": "",
    "bsnRgsNo_2": "",     "bsnRgsNo_3": "",
    "crpRgsNo": "",     "textCrpNm": company,
    "corporationType": "all"
    }
    r = requests.post(text_url, data=payload)
    bs = BeautifulSoup(r.text)
    result = {x.text.strip():p.findall(x['href'])[0] for x in bs.find_all('a')[:-1]}
    return result[company]


get_ticker("삼성전자")


In [ ]:
def get_statement(company, company_code, start_date, end_date):
    data = {
    "currentPage": 1,     "maxResults": 15,
    "maxLinks": 10,     "sort": "date",
    "series": "desc",     "textCrpCik": company_code,
    "lateKeyword": "",    "keyword": "",     "reportNamePopYn": "",
    "textkeyword": "",     "businessCode": "all",     "autoSearch": "N",
    "option": "corp",
    "textCrpNm": company,
    "reportName": "",    "tocSrch": "",
    "textPresenterNm": "",
    "startDate": start_date,
    "endDate":  end_date,
    "decadeType": "",    "finalReport": "recent",    "businessNm": "전체",
    "corporationType": "all",    "closingAccountsMonth": "all",
    "reportName2": "",    "tocSrch2": "",
    "publicType": [
        "A001", "A002", "A003", "A005", "A004",
        "B001", "B002", "B003",
        "C001", "C002", "C003", "C004", "C005", "C006", "C007", "C008", "C009", "C010", "C011",
        "D001", "D004", "D003", "D002", "D005",
        "E001", "E002", "E003", "E004", "E005", "E006", "E007", "E008", "E009",
        "F001", "F002", "F003", "F004", "F005",
        "G001", "G002", "G003",
        "H001", "H002", "H003", "H004", "H005", "H006"
        ]
    }
    url = "https://dart.fss.or.kr/dsab007/detailSearch.ax"
    return requests.post(url, data=data).text



rt = get_statement("삼성전자", "00126380", "20240101" , '20250731')



In [ ]:
from langchain.agents import AgentType, initialize_agent, load_tools, Tool
from langchain.chat_models import ChatOpenAI
from langchain.tools import BaseTool


from pydantic import BaseModel, Field
class GetStatementInput(BaseModel):
    company: str = Field(..., description='회사 이름')
    company_code: str = Field(..., description='회사 코드')
    start_date: str = Field(..., description="YYYYMMDD 형식의 시작 날짜")
    end_date: str = Field(..., description="YYYYMMDD 형식의 종료 날짜")




from typing import Optional, Type
class GetStatementTool(BaseTool):
    name: str = 'GetStatementTool'            
    description: str = '기업의 공시 자료를 가져옵니다.'
   
    args_schema: Type[BaseModel] = GetStatementInput


    def _run(self, company, company_code, start_date, end_date):
        return get_statement(company=company, company_code=company_code, start_date=start_date, end_date=end_date)

class GetTickerTool(BaseTool):
    name: str = 'GetTickerTool'                
    description: str = 'Dart에서 사용하는 기업의 code 리턴 '


    def _run(self, company):
        return get_ticker(company=company)


tools = [GetTickerTool(), GetStatementTool()]
chat = ChatOpenAI(model = 'gpt-4o-2024-08-06')
agent = initialize_agent(
    tools,
    chat,
    agent=AgentType.OPENAI_FUNCTIONS,
    verbose=True
)


result = agent.run("당신은 지금부터 애널리스트입니다. LG전자의 20240101부터 20241231까지 공시 정보를 활용해서 분석 리포트를 자세히 작성할 것. 그리고 투자 의견에 대한 내용도 결론에 넣을 것")
